In [1]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

# import necessary libraries
import numpy as np
import pandas as pd
import xarray as xr

In [2]:
# Open the dataset
path = "/home/jatin/Downloads/sst1970-1979.nc"
ds = xr.open_dataset(path)

print(ds)

<xarray.Dataset>
Dimensions:  (lat: 180, lon: 360, time: 120)
Coordinates:
  * lon      (lon) float32 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
  * lat      (lat) float32 89.5 88.5 87.5 86.5 85.5 ... -86.5 -87.5 -88.5 -89.5
  * time     (time) datetime64[ns] 1970-01-01T12:00:00 ... 1979-12-01T12:00:00
Data variables:
    sst      (time, lat, lon) float32 ...
Attributes:
    Conventions:                CF-1.0
    title:                      Monthly version of HadISST sea surface temper...
    institution:                Met Office, Hadley Centre for Climate Research
    source:                     HadISST
    history:                    09/11/2006 HadISST converted to NetCDF from p...
    references:                 Rayner, N. A., Parker, D. E., Horton, E. B., ...
    comment:                    Data restrictions: for academic research use ...
    supplementary_information:  Updates and supplementary information will be...
    contact:                    john.kennedy@metoffice.g

In [3]:
# Extract the data variable
sst = ds['sst']
print(sst)

<xarray.DataArray 'sst' (time: 120, lat: 180, lon: 360)>
[7776000 values with dtype=float32]
Coordinates:
  * lon      (lon) float32 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
  * lat      (lat) float32 89.5 88.5 87.5 86.5 85.5 ... -86.5 -87.5 -88.5 -89.5
  * time     (time) datetime64[ns] 1970-01-01T12:00:00 ... 1979-12-01T12:00:00
Attributes:
    long_name:     Monthly 1 degree resolution SST
    units:         degrees C
    actual_range:  [-1000.         33.53844]
    description:   HadISST sea surface temperature. values of -1000 indicate ...


In [114]:
def month_to_season(xMon, SEASON):
    
    # xMon is the DataArray and SEASON is the season passed as an argument
    
    # Create a new array of seasons
    season = np.array(["DJF","JFM","FMA","MAM","AMJ","MJJ","JJA","JAS","ASO","SON","OND","NDJ"])
    
    # Check if the season argument passed is valid or not
    if SEASON not in season:
        print("month_to_season: bad season: SEASON={}".format(SEASON))
        return None
    else:
        NMO_tup = np.where(season==SEASON)
        NMO = NMO_tup[0].item()
    
    # Get the rank of dataarray
    len_of_dim = xMon.sizes
    rank_of_dim = len(len_of_dim)
    
    # Check if the rank of dim is valid or invalid
    if (rank_of_dim == 2 or rank_of_dim >= 5):
        print("month_to_season12: rank = {}".format(rank_of_dim))
        print("----- rank currently not handled -----")
        return None
    
    # Check if number of months are multiple of 12
    no_of_months = 12
    no_of_time = len_of_dim[xMon.dims[0]]
    if ((no_of_time % no_of_months) != 0):
        print("month_to_season12: dimension must be a multiple of 12")
        return None
   
    
    # Store the size of latitude and longitude
    if (rank_of_dim >= 3):
        size_of_lat = len_of_dim[xMon.dims[rank_of_dim-2]]
        size_of_lon = len_of_dim[xMon.dims[rank_of_dim-1]]
    
    # Calculate number of years
    no_of_years = int(no_of_time/no_of_months)
    con = 1./3.
    
    # Starting and ending years
    no_year_start = 0
    no_year_last = no_of_years - 1
    if (NMO == 0):
        no_year_start = 1
    elif (NMO == (no_of_months-1)):
        no_year_last = no_of_years - 2
    
    # Create a copy of new dataarray using the existing one
    da_new = xMon.copy(deep = True, data = None)
    
    # For rank = 1
    if (rank_of_dim == 1):
        for nyr in range(no_year_start, (no_year_last+1)):
            n = nyr * no_of_months + NMO
            da_new[n] = (xMon[n-1] + xMon[n] + xMon[n+1]) * con
        
        if (NMO == 0):
            n = 0
            da_new[n] = (xMon[n] + xMon[n+1])*0.5
        
        if (NMO == (nmos-1)):
            n = (no_of_years-1)*no_of_months + NMO
            da_new[n] = (xMon[n] + xMon[n-1])*0.5

        da_new = da_new[NMO:no_of_time:no_of_months]               
        
    # For rank = 3
    if (rank_of_dim == 3):
        for nyr in range(no_year_start, (no_year_last+1)):
            n = nyr*no_of_months + NMO
            da_new[n,:,:] = (xMon[(n-1),:,:] + xMon[n,:,:] + xMon[(n+1),:,:]) * con
        
        if (NMO == 0):
            n = 0
            da_new[n,:,:] = (xMon[n,:,:] + xMon[(n+1),:,:])*0.5
        
        if (NMO == (no_of_months-1)):
            n = (no_of_years-1)*no_of_months + NMO
            da_new[n,:,:] = (xMon[n,:,:] + xMon[(n-1),:,:])*0.5
        
        da_new = da_new[NMO:no_of_time:no_of_months,:,:]

    
    # For rank = 4
    if (rank_of_dim == 4):
        for nyr in range(no_year_start, (no_year_last+1)):
            n = nyr*no_of_months + NMO
            da_new[n,:,:,:] = (xMon[(n-1),:,:,:] + xMon[n,:,:,:] + xMon[(n+1),:,:,:]) * con
        
        if (NMO == 0):
            n = 0
            da_new[0,:,:,:] = (xMon[n,:,:,:] + xMon[(n+1),:,:,:])*0.5
        
        if (NMO == (nmos-1)):
            n = (no_of_years-1)*no_of_months + NMO
            da_new[(n),:,:,:] = (xMon[n,:,:,:] + xMon[(n-1),:,:,:])*0.5
        
        da_new = da_new[NMO:no_of_time:no_of_months,:,:,:]
                                  
        
    # Add and update the attributes
    da_new = da_new.rename("x"+SEASON)
    da_new.attrs['NMO'] = NMO
    da_new.attrs['long_name'] = SEASON + ": " + da_new.attrs['long_name']
      
    # Return the newly created DataArray
    return da_new
    

In [115]:
xJJA = month_to_season(sst,"JJA")

In [116]:
xJJA

<xarray.DataArray 'xJJA' (time: 10, lat: 180, lon: 360)>
array([[[ -667.2667 ,  -334.53333, ...,  -667.2667 ,  -667.2667 ],
        [   -1.8    ,    -1.8    , ...,    -1.8    ,    -1.8    ],
        ...,
        [        nan,         nan, ...,         nan,         nan],
        [        nan,         nan, ...,         nan,         nan]],

       [[ -667.2667 ,  -334.53333, ...,  -667.2667 ,  -667.2667 ],
        [   -1.8    ,    -1.8    , ...,    -1.8    ,    -1.8    ],
        ...,
        [        nan,         nan, ...,         nan,         nan],
        [        nan,         nan, ...,         nan,         nan]],

       ...,

       [[ -667.2667 ,  -334.53333, ...,  -667.2667 ,  -667.2667 ],
        [   -1.8    ,    -1.8    , ...,    -1.8    ,    -1.8    ],
        ...,
        [        nan,         nan, ...,         nan,         nan],
        [        nan,         nan, ...,         nan,         nan]],

       [[-1000.     , -1000.     , ..., -1000.     , -1000.     ],
        [ -334